# Multi-Agent System Demo

This notebook demonstrates the multi-agent system with collaborative AI agents.

## Setup

First, let's import the necessary modules and set up logging.

In [ ]:
import asyncio
import json
import logging
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from agents.manager.coordinator import ManagerAgent
from agents.research.gatherer import ResearchAgent
from agents.writer.creator import WriterAgent
from agents.analyzer.insights import AnalyzerAgent
from communication.message_bus import MessageBus

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ Imports successful")

## Initialize the Multi-Agent System

Create the message bus and all agents.

In [ ]:
# Create message bus
message_bus = MessageBus()

# Create agents
manager = ManagerAgent(message_bus)
research_agent = ResearchAgent(message_bus)
writer_agent = WriterAgent(message_bus)
analyzer_agent = AnalyzerAgent(message_bus)

print("✓ Agents created")

In [ ]:
# Start all agents
await manager.start()
await research_agent.start()
await writer_agent.start()
await analyzer_agent.start()

print(f"✓ Active agents: {message_bus.get_registered_agents()}")

## Example 1: Manager Delegation

The manager agent decomposes tasks and delegates to specialists.

In [ ]:
result = await manager.delegate_task(
    description="Research and write a report on renewable energy",
    parameters={
        "topic": "renewable energy trends",
        "needs_research": True
    }
)

print(json.dumps(result, indent=2, default=str))

## Example 2: Direct Research Query

Query the research agent directly.

In [ ]:
research_task = {
    "task_id": "research_001",
    "description": "Research machine learning",
    "parameters": {"query": "machine learning applications"}
}

research_result = await research_agent.process_task(research_task)
print(json.dumps(research_result, indent=2, default=str))

## Example 3: Data Analysis

Analyze data using the analyzer agent.

In [ ]:
analysis_task = {
    "task_id": "analysis_001",
    "description": "Analyze research data",
    "parameters": {"data": research_result}
}

analysis_result = await analyzer_agent.process_task(analysis_task)
print(json.dumps(analysis_result, indent=2, default=str))

## Example 4: Content Writing

Generate content using the writer agent.

In [ ]:
content = await writer_agent.create_content(
    topic="Artificial Intelligence Ethics",
    content_type="article",
    style="formal"
)

print(content)

## Message History

View the communication between agents.

In [ ]:
history = message_bus.get_history(limit=10)

print(f"Total messages: {len(message_bus.get_history())}")
print("\nRecent messages:")
for msg in history[-10:]:
    print(f"  {msg.from_:15} -> {msg.to:15}: {msg.message_type.value}")

## Agent Status

Check the status of all agents.

In [ ]:
agents = [manager, research_agent, writer_agent, analyzer_agent]

for agent in agents:
    status = agent.get_status()
    print(f"\n{status.agent_id}:")
    print(f"  State: {status.state}")
    print(f"  Capabilities: {', '.join(status.capabilities)}")
    print(f"  Workload: {status.workload * 100:.0f}%")

## Cleanup

Stop all agents.

In [ ]:
await manager.stop()
await research_agent.stop()
await writer_agent.stop()
await analyzer_agent.stop()

print("✓ All agents stopped")